In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nbaiot-dataset/7.gafgyt.combo.csv
/kaggle/input/nbaiot-dataset/9.gafgyt.combo.csv
/kaggle/input/nbaiot-dataset/5.gafgyt.combo.csv
/kaggle/input/nbaiot-dataset/1.mirai.udp.csv
/kaggle/input/nbaiot-dataset/4.gafgyt.udp.csv
/kaggle/input/nbaiot-dataset/6.gafgyt.udp.csv
/kaggle/input/nbaiot-dataset/6.gafgyt.junk.csv
/kaggle/input/nbaiot-dataset/data_summary.csv
/kaggle/input/nbaiot-dataset/5.gafgyt.udp.csv
/kaggle/input/nbaiot-dataset/9.gafgyt.junk.csv
/kaggle/input/nbaiot-dataset/9.mirai.scan.csv
/kaggle/input/nbaiot-dataset/1.benign.csv
/kaggle/input/nbaiot-dataset/2.mirai.udpplain.csv
/kaggle/input/nbaiot-dataset/3.gafgyt.combo.csv
/kaggle/input/nbaiot-dataset/4.gafgyt.combo.csv
/kaggle/input/nbaiot-dataset/6.mirai.scan.csv
/kaggle/input/nbaiot-dataset/5.mirai.udp.csv
/kaggle/input/nbaiot-dataset/3.benign.csv
/kaggle/input/nbaiot-dataset/3.gafgyt.junk.csv
/kaggle/input/nbaiot-dataset/7.gafgyt.scan.csv
/kaggle/input/nbaiot-dataset/features.csv
/kaggle/input/nbaiot-dataset/6

In [2]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# --- Configuration Constants ---
# NOTE: Update this path if your dataset is named differently.
BASE_PATH = '../input/nbaiot-dataset/' 
RANDOM_SEED = 42         # For reproducible results
SAMPLE_SIZE = 10000      # Your requested random sample size

# List to hold DataFrames from all files
all_data = []

print("Setup Complete. Ready for Step 1: Merging.")

Setup Complete. Ready for Step 1: Merging.


In [3]:
# ==============================================================================
# 1. Merge & Labeling (Using Library Functions, No Manual Loops)
# ==============================================================================

import glob
import pandas as pd
import os

print("Step 1: Merging Data and Creating Labels...")

# Get all CSV file paths (recursive search)
file_paths = glob.glob(os.path.join(BASE_PATH, "**", "*.csv"), recursive=True)

if not file_paths:
    raise FileNotFoundError("No CSV files found. Check BASE_PATH.")

# Read all CSV files into list of DataFrames
df_list = [pd.read_csv(fp) for fp in file_paths]

# Concatenate all data into one DataFrame
df_raw = pd.concat(df_list, ignore_index=True)

# Create label based on filename (vectorized)
df_raw['label'] = [
    0 if 'benign' in os.path.basename(fp).lower() else 1
    for fp in file_paths
    for _ in range(len(pd.read_csv(fp)))
]

print(f"✅ Files merged successfully. Total rows: {len(df_raw)}")


Step 1: Merging Data and Creating Labels...
✅ Files merged successfully. Total rows: 7062819


In [4]:
# ==============================================================================
# 2. Sampling: Extract 10,000 Rows (Using Library Function - No Loops)
# ==============================================================================

from sklearn.utils import resample   # ✅ Library Sampling Function

print("\nStep 2: Sampling the Dataset...")

# SAMPLE_SIZE and RANDOM_SEED must be already defined.
# Example:
# SAMPLE_SIZE = 10000
# RANDOM_SEED = 42

# If dataset is smaller than sample size, resample will return all rows automatically
df_sampled = resample(
    df_raw,
    replace=False,           # Do NOT duplicate rows
    n_samples=min(SAMPLE_SIZE, len(df_raw)),  # Auto-adjust sample size
    random_state=RANDOM_SEED
)

print(f"✅ Data sampled successfully: {len(df_sampled)} rows.")



Step 2: Sampling the Dataset...
✅ Data sampled successfully: 10000 rows.


In [5]:
# ==============================================================================
# 3. Data Cleaning: Handle Special Numerical Values (Inf and NaN) - Corrected
# ==============================================================================

import numpy as np
import pandas as pd

print("\nStep 3: Handling Special Values (Inf and NaN)...")

# Replace positive and negative infinite values with large numbers
df_sampled = df_sampled.replace([np.inf, -np.inf], [1e9, -1e9])

# Fill NaN values with 0 safely using pandas function
df_sampled = df_sampled.fillna(0)

# Optional: ensure correct dtypes to avoid future warnings
df_sampled = df_sampled.infer_objects()

# Separate features (X) and target (Y)
Y = df_sampled['label']
X = df_sampled.drop(columns=['label'])

print("✅ Special values handled successfully.")
print("Data is now separated into:")
print(f"  X (Features): {X.shape}")
print(f"  Y (Labels): {Y.shape}")



Step 3: Handling Special Values (Inf and NaN)...
✅ Special values handled successfully.
Data is now separated into:
  X (Features): (10000, 122)
  Y (Labels): (10000,)


/tmp/ipykernel_13/1021418769.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_sampled = df_sampled.replace([np.inf, -np.inf], [1e9, -1e9])


In [6]:
# ==============================================================================
# 4. Feature Scaling: Apply Min-Max Normalization (Using Library)
# ==============================================================================

from sklearn.preprocessing import MinMaxScaler   # ✅ Library Used
import pandas as pd                              # ✅ Needed to convert back to DataFrame

print("\nStep 4: Applying Min-Max Scaling (Normalization)...")

# Initialize Min-Max Scaler
scaler = MinMaxScaler()

# Fit the scaler on the feature data and transform it
X_scaled = scaler.fit_transform(X)

# Convert scaled array back to DataFrame
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

print("✅ Min-Max Scaling Complete.")
print(f"Final Preprocessed Features (X_scaled_df): {X_scaled_df.shape}")



Step 4: Applying Min-Max Scaling (Normalization)...
✅ Min-Max Scaling Complete.
Final Preprocessed Features (X_scaled_df): (10000, 122)


In [7]:
# ==============================================================================
# 5. Final Split: Separate into Training and Testing Sets (Using Library)
# ==============================================================================

from sklearn.model_selection import train_test_split   # ✅ Library Used

print("\nStep 5: Splitting Data for Training and Testing...")

# Split the data using 70/30 ratio with stratification
X_train, X_test, Y_train, Y_test = train_test_split(
    X_scaled_df,         # Features after scaling
    Y,                   # Target labels
    test_size=0.3,       # 30% data for testing
    random_state=RANDOM_SEED,
    stratify=Y           # ✅ Keeps anomaly ratio same in train & test
)

print("✅ Full Preprocessing Pipeline Complete.")
print(f"  Training Features (X_train): {X_train.shape}")
print(f"  Testing Features (X_test): {X_test.shape}")
print("You can now proceed to Model Training using X_train and Y_train.")



Step 5: Splitting Data for Training and Testing...
✅ Full Preprocessing Pipeline Complete.
  Training Features (X_train): (7000, 122)
  Testing Features (X_test): (3000, 122)
You can now proceed to Model Training using X_train and Y_train.
